For the google data I wanted to load and plot on orthographic Earth, sunlocked. 

Load and save day of google data as a parquet. 

In [ ]:
import pandas as pd
import s2sphere as s2

# --- Load the compressed CSV directly ---
csv_path = 'ionosphere_central/google_ground/data/vtec_maps/vtec_2024_04_10.csv.gz'
with open(csv_path, 'rb') as f:
    raw_one_day_vtec_df = pd.read_csv(f, compression='gzip')

# --- Add lat/lon and timestamp columns ---
def set_extra_vtec_df_columns(vtec_df):
    vtec_df['timestamp'] = pd.to_datetime(vtec_df['utc_sec'], unit='s', utc=True)
    vtec_df['date_and_time'] = vtec_df['timestamp'].dt.strftime('%Y_%m_%d_T_%H_%M_%S')
    s2_cell_ids = [s2.CellId.from_token(x) for x in vtec_df['pierce_s2_token']]
    latlngs = [c.to_lat_lng() for c in s2_cell_ids]
    vtec_df['pierce_lat'] = [latlng.lat().degrees for latlng in latlngs]
    vtec_df['pierce_lng'] = [latlng.lng().degrees for latlng in latlngs]
    keep_cols = ['timestamp', 'pierce_lat', 'pierce_lng', 'vtec']
    return vtec_df[keep_cols].copy()

# --- Convert and save as Parquet ---
one_day_vtec_df = set_extra_vtec_df_columns(raw_one_day_vtec_df)
parquet_path = 'vtec_2024_04_10_for_plotting.parquet'
one_day_vtec_df.to_parquet(parquet_path)
print(f"Saved {parquet_path}")
print(one_day_vtec_df.head())


Now plot load the parquet and plot on orthographic Earth, that rotate as if the sun is looking at it. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib.animation as animation

# --- Load processed VTEC pierce point data ---
one_day_vtec_df = pd.read_parquet("vtec_2024_04_10_for_plotting.parquet")
one_day_vtec_df['timestamp'] = pd.to_datetime(one_day_vtec_df['timestamp'], utc=True)
all_times = sorted(one_day_vtec_df['timestamp'].unique())

vtec_vmin = one_day_vtec_df['vtec'].quantile(0.02)
vtec_vmax = one_day_vtec_df['vtec'].quantile(0.98)

def get_subsolar_lon(dt):
    seconds = dt.hour * 3600 + dt.minute * 60 + dt.second
    frac_of_day = seconds / 86400.0
    return (frac_of_day * 360.0) - 180.0

fig = plt.figure(figsize=(16, 8))

def update(frame_idx):
    plt.clf()
    ts = all_times[frame_idx]
    df_now = one_day_vtec_df[one_day_vtec_df['timestamp'] == ts]
    subsolar_lon = get_subsolar_lon(ts)
    nightside_lon = (subsolar_lon + 180) % 360
    if nightside_lon > 180:
        nightside_lon -= 360

    # --- Sun-facing side ---
    ax1 = plt.subplot(1, 2, 1, projection=ccrs.Orthographic(central_longitude=-subsolar_lon, central_latitude=0))
    ax1.set_global()
    ax1.coastlines()
    sc1 = ax1.scatter(
        df_now['pierce_lng'], df_now['pierce_lat'], c=df_now['vtec'], s=12,
        cmap='gist_ncar', vmin=vtec_vmin, vmax=vtec_vmax, alpha=0.8,
        transform=ccrs.PlateCarree(), edgecolor='none'
    )
    ax1.set_title(f"Sun-facing Side\nSubsolar Lon: {subsolar_lon:.1f}°", fontsize=12)

    # --- Dark side ---
    ax2 = plt.subplot(1, 2, 2, projection=ccrs.Orthographic(central_longitude=-nightside_lon, central_latitude=0))
    ax2.set_global()
    ax2.coastlines()
    sc2 = ax2.scatter(
        df_now['pierce_lng'], df_now['pierce_lat'], c=df_now['vtec'], s=12,
        cmap='gist_ncar', vmin=vtec_vmin, vmax=vtec_vmax, alpha=0.8,
        transform=ccrs.PlateCarree(), edgecolor='none'
    )
    ax2.set_title(f"Dark Side\nOpposite Lon: {nightside_lon:.1f}°", fontsize=12)

    # --- Shared colorbar below both ---
    cbar_ax = fig.add_axes([0.25, 0.10, 0.5, 0.03])
    plt.colorbar(sc1, cax=cbar_ax, orientation='horizontal', label='VTEC (TECU)')
    fig.suptitle(f'VTEC | {ts.strftime("%Y-%m-%d %H:%M:%S UTC")}', fontsize=15)
    return sc1, sc2

ani = animation.FuncAnimation(fig, update, frames=len(all_times), interval=200, blit=False)
ani.save('vtec_orthographic_day_night.gif', writer='pillow', fps=6)
print("Saved vtec_orthographic_day_night.gif")

# --- For Jupyter inline display ---
from IPython.display import HTML
plt.rcParams['animation.embed_limit'] = 50_000_000
HTML(ani.to_jshtml())
